# Train Your DreamBooth Model

In [ ]:
%cd /home/ubuntu/dreambooth
SD_NAME="CompVis/stable-diffusion-v1-4" # If you downloaded our model cache, then set it to where it is. e.g. "/home/models--CompVis--stable-diffusion-v1-4/snapshots/114c79c111c5836a89768787ba5b4ceee25bb734"

INPUT_DIR="/home/ubuntu/data" # This is the path to input training photos

!rm -rf "${INPUT_DIR}/.ipynb_checkpoints" # Some times notebook saves checkpoints in the data and that causes problems.

In [ ]:
TOKEN="aabbccddeeffgg"
INSTANCE_PROMPT="a photo of " + TOKEN + " person"

In [ ]:
# memory effeicient attention + fp16
# This works on a single A10
MODEL_DIR="/home/ubuntu/model_mea_fp16"
!mkdir -p "$MODEL_DIR"

MAX_NUM_STEPS=500
BATCH_SIZE=2
LR=1e-6
RESOLUTION=512

!(accelerate launch  --gpu_ids 0 train_dreambooth.py \
  --pretrained_model_name_or_path=$SD_NAME \
  --train_text_encoder \
  --center_crop \
  --instance_data_dir=$INPUT_DIR \
  --output_dir=$MODEL_DIR \
  --instance_prompt="$INSTANCE_PROMPT" \
  --resolution=$RESOLUTION \
  --train_batch_size=$BATCH_SIZE \
  --gradient_checkpointing \
  --learning_rate=$LR \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=$MAX_NUM_STEPS \
  --enable_xformers_memory_efficient_attention \
  --mixed_precision=fp16 \
)
    

In [ ]:
# Inference (results will be saved to PRED_DIR)
MODEL_DIR="/home/ubuntu/model_mea_fp16"
PRED_DIR="/home/ubuntu/output_mea_fp16" # This is the path to save the predicted images
!mkdir -p "$PRED_DIR"

NUM_PRED=1 # number of predictions per prompt
!( python test_dreambooth.py \
    --model_path $MODEL_DIR \
    --pred_path $PRED_DIR \
    --num_preds $NUM_PRED \
    --ddim \
    --token $TOKEN \
)

import glob
from IPython.display import Image, display
for imageName in glob.glob(PRED_DIR +'/*.png'):
    print(imageName)
    display(Image(filename=imageName))

In [ ]:
# Inference (results will be saved to PRED_DIR)
MODEL_DIR="/home/ubuntu/model_wandb_nsteps"
PRED_DIR="/home/ubuntu/output_wandb_nsteps" # This is the path to save the predicted images
!mkdir -p "$PRED_DIR"

NUM_PRED=1 # number of predictions per prompt
!( python test_dreambooth.py \
    --model_path $MODEL_DIR \
    --pred_path $PRED_DIR \
    --num_preds $NUM_PRED \
    --ddim \
    --token $TOKEN \
)

import glob
from IPython.display import Image, display
for imageName in glob.glob(PRED_DIR +'/*.png'):
    print(imageName)
    display(Image(filename=imageName))